In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
def print_along_lines(series_slice):
    for (ind, line) in zip(series_slice.index, series_slice):
        print(f"{ind:3d} {parsed_dates[ind]} {series_slice[ind]}")

In [2]:
doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 500, dtype: object

In [3]:
# a dictionary to collect all parsed dates.

parsed_dates = dict.fromkeys(df.index)

In [4]:
# all numeric date

all_numeric = df[:125]

for (ind, date) in zip(all_numeric.index, all_numeric.str.findall(r"[\d]{1,2}[/-]{1}[\d]{1,2}[/-]{1}[\d]{2,4}")):
        # parsed_dates[ind] = date[0][:-2] + "19" + date[0][-2:]
        _date = date[0]
        # print(_date)
        if "-" not in _date:
            if len(_date.split("/")[2]) < 4:      
                _strdate = _date[:-2] + "19" + _date[-2:]
                # print(f"{_date:20s} {_strdate:20s}")
                parsed_dates[ind] = datetime.strptime(_strdate, "%m/%d/%Y")
            else:
                # print(f"{_date:20s}")
                parsed_dates[ind] = datetime.strptime(_date, "%m/%d/%Y")
                
        elif "-" in _date:
            if len(_date.split("-")[2]) < 4:
                _strdate = _date[:-2] + "19" + _date[-2:]
                # print(f"{_date:20s} {_strdate:20s}")
                parsed_dates[ind] = datetime.strptime(_strdate, "%m-%d-%Y")
            else:
                # print(f"{_date:20s}")
                parsed_dates[ind] = datetime.strptime(_date, "%m-%d-%Y")

In [5]:
# print_along_lines(all_numeric)

In [6]:
month_in_words = df[125:194]

for (ind, date) in zip(month_in_words.index, month_in_words.str.findall(r"[\d]{1,2} [\w]{3,} [\d]{2,4}")):
        # parsed_dates[ind] = date[0]
        _strdate  = date[0]
        try:
            parsed_dates[ind] = datetime.strptime(_strdate, "%d %B %Y")
        except ValueError:
            try:
                parsed_dates[ind] = datetime.strptime(_strdate, "%d %b %Y")
            except:
                print("FUCK")

In [7]:
start_with_month = df[194:228]

for (ind, date) in zip(start_with_month.index, start_with_month.str.findall(r"[\w]{3,}[.,]* [\d]+[,]* [\d]{2,4}")):
        # parsed_dates[ind] = date[0]
        _strdate = date[0].replace(",", "").replace(".", "")
        try:
            parsed_dates[ind] = datetime.strptime(_strdate, "%B %d %Y")
        except ValueError:
            try:
                parsed_dates[ind] = datetime.strptime(_strdate, "%b %d %Y")
            except:
                print("FUCK")

In [8]:
parsed_dates;

In [9]:
month_and_year_only = df[228:343]
for (ind, date) in zip(month_and_year_only.index, month_and_year_only.str.findall(r"[A-Z]{1}[a-z]{2,}[,]* [\d]{4}")):
    # parsed_dates[ind] = "01 " + date[0]
    _strdate = "01 " + date[0].replace(",", "").replace("Janaury", "January").replace("Decemeber", "December")
    try:
        parsed_dates[ind] = datetime.strptime(_strdate, "%d %B %Y")
    except ValueError:
        try:
            parsed_dates[ind] = datetime.strptime(_strdate, "%d %b %Y")
        except:
            print("FUCK", _strdate)

In [10]:
parsed_dates;

In [11]:
month_and_year_numeric = df[343:455]
for (ind, date) in zip(month_and_year_numeric.index, month_and_year_numeric.str.findall(r"[\d]{1,2}/[\d]{2,4}")):
    # parsed_dates[ind] = "01/" + date[0]
    _strdate = "01/" + date[0]
    parsed_dates[ind] = datetime.strptime(_strdate, "%d/%m/%Y")

In [12]:
parsed_dates;

In [13]:
year_only = df[455:]
for (ind, date) in zip(year_only.index, year_only.str.findall(r"[\d]{4}")):
    # parsed_dates[ind] =  "01/01/" + date[0]
    _strdate = "01/01/" + date[0]
    parsed_dates[ind] = datetime.strptime(_strdate, "%d/%m/%Y")

In [24]:
dates = pd.Series(parsed_dates, name = "date")

In [26]:
dates.duplicated().sum()

52

In [27]:
# There are 52 duplicates!

dates.shape, len(set(dates))

((500,), 448)

In [34]:
dates.reset_index(drop = False).sort_values(["date", "index"]).index

Int64Index([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98,
            ...
            152, 235, 464, 253, 231, 427, 141, 186, 161, 413],
           dtype='int64', length=500)

In [16]:
sorted_dates = dates.sort_values()

In [17]:
pd.Series(sorted_dates.index)

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [18]:
len(sorted_dates.index), len(sorted_dates.index.unique())

(500, 500)

In [19]:
with open("medical.csv", "w") as handle:
    string = ""
    
    for (ind, date, line) in zip(parsed_dates.keys(), parsed_dates.values(), df):
        string += f"{ind}; {date.strftime('%d %m %Y')}; {line}"
    
    print(string, file = handle)

In [79]:
parsed_dates

{0: datetime.datetime(1993, 3, 25, 0, 0),
 1: datetime.datetime(1985, 6, 18, 0, 0),
 2: datetime.datetime(1971, 7, 8, 0, 0),
 3: datetime.datetime(1975, 9, 27, 0, 0),
 4: datetime.datetime(1996, 2, 6, 0, 0),
 5: datetime.datetime(1979, 7, 6, 0, 0),
 6: datetime.datetime(1978, 5, 18, 0, 0),
 7: datetime.datetime(1989, 10, 24, 0, 0),
 8: datetime.datetime(1986, 3, 7, 0, 0),
 9: datetime.datetime(1971, 4, 10, 0, 0),
 10: datetime.datetime(1985, 5, 11, 0, 0),
 11: datetime.datetime(1975, 4, 9, 0, 0),
 12: datetime.datetime(1998, 8, 1, 0, 0),
 13: datetime.datetime(1972, 1, 26, 0, 0),
 14: datetime.datetime(1990, 5, 24, 0, 0),
 15: datetime.datetime(2011, 1, 25, 0, 0),
 16: datetime.datetime(1982, 4, 12, 0, 0),
 17: datetime.datetime(1976, 10, 13, 0, 0),
 18: datetime.datetime(1998, 4, 24, 0, 0),
 19: datetime.datetime(1977, 5, 21, 0, 0),
 20: datetime.datetime(1998, 7, 21, 0, 0),
 21: datetime.datetime(1979, 10, 21, 0, 0),
 22: datetime.datetime(1990, 3, 3, 0, 0),
 23: datetime.datetime(19

In [35]:
def date_sorter():
    
    parsed_dates = dict.fromkeys(df.index)
    
    all_numeric = df[:125]
    for (ind, date) in zip(all_numeric.index, all_numeric.str.findall(r"[\d]{1,2}[/-]{1}[\d]{1,2}[/-]{1}[\d]{2,4}")):

        _date = date[0]

        if "-" not in _date:
            if len(_date.split("/")[2]) < 4:      
                _strdate = _date[:-2] + "19" + _date[-2:]

                parsed_dates[ind] = datetime.strptime(_strdate, "%m/%d/%Y")
            else:

                parsed_dates[ind] = datetime.strptime(_date, "%m/%d/%Y")
                
        elif "-" in _date:
            if len(_date.split("-")[2]) < 4:
                _strdate = _date[:-2] + "19" + _date[-2:]

                parsed_dates[ind] = datetime.strptime(_strdate, "%m-%d-%Y")
            else:

                parsed_dates[ind] = datetime.strptime(_date, "%m-%d-%Y")
    
    month_in_words = df[125:194]
    for (ind, date) in zip(month_in_words.index, month_in_words.str.findall(r"[\d]{1,2} [\w]{3,} [\d]{2,4}")):
            # parsed_dates[ind] = date[0]
            _strdate  = date[0]
            try:
                parsed_dates[ind] = datetime.strptime(_strdate, "%d %B %Y")
            except ValueError:
                try:
                    parsed_dates[ind] = datetime.strptime(_strdate, "%d %b %Y")
                except:
                    print("FUCK")
                    
    start_with_month = df[194:228]
    for (ind, date) in zip(start_with_month.index, start_with_month.str.findall(r"[\w]{3,}[.,]* [\d]+[,]* [\d]{2,4}")):
            # parsed_dates[ind] = date[0]
            _strdate = date[0].replace(",", "").replace(".", "")
            try:
                parsed_dates[ind] = datetime.strptime(_strdate, "%B %d %Y")
            except ValueError:
                try:
                    parsed_dates[ind] = datetime.strptime(_strdate, "%b %d %Y")
                except:
                    print("FUCK")
    
    month_and_year_only = df[228:343]
    for (ind, date) in zip(month_and_year_only.index, month_and_year_only.str.findall(r"[A-Z]{1}[a-z]{2,}[,]* [\d]{4}")):
        # parsed_dates[ind] = "01 " + date[0]
        _strdate = "01 " + date[0].replace(",", "").replace("Janaury", "January").replace("Decemeber", "December")
        try:
            parsed_dates[ind] = datetime.strptime(_strdate, "%d %B %Y")
        except ValueError:
            try:
                parsed_dates[ind] = datetime.strptime(_strdate, "%d %b %Y")
            except:
                print("FUCK", _strdate)
    
    month_and_year_numeric = df[343:455]
    for (ind, date) in zip(month_and_year_numeric.index, month_and_year_numeric.str.findall(r"[\d]{1,2}/[\d]{2,4}")):
        # parsed_dates[ind] = "01/" + date[0]
        _strdate = "01/" + date[0]
        parsed_dates[ind] = datetime.strptime(_strdate, "%d/%m/%Y")
        
    year_only = df[455:]
    for (ind, date) in zip(year_only.index, year_only.str.findall(r"[\d]{4}")):
        # parsed_dates[ind] =  "01/01/" + date[0]
        _strdate = "01/01/" + date[0]
        parsed_dates[ind] = datetime.strptime(_strdate, "%d/%m/%Y")
        
    dates = pd.Series(parsed_dates, name = "date")
    return pd.Series(dates.reset_index(drop = False).sort_values(["date", "index"]).index)

In [36]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64